In [6]:
import crypten
import crypten.nn as cnn
import crypten.mpc as mpc
import crypten.communicator as comm


import torch
import torch.nn as nn

from torchvision.models.resnet import *

crypten.init()
torch.set_num_threads(1)

/home/comet/anaconda3/envs/research_py/lib/python3.9/site-packages/crypten/__init__.py:64: RuntimeWarning: CrypTen is already initialized.
  warnings.warn("CrypTen is already initialized.", RuntimeWarning)


In [ ]:

x = torch.tensor([1.0, 2.0, 3.0])
x_enc = crypten.cryptensor(x)


y = torch.tensor([3.0, 4.0, 5.0])
y_enc = crypten.cryptensor(y)

In [ ]:
# has to be encoding first then the non-encoded value

z = y_enc + x
z_enc = x_enc + y_enc

crypten.print(z.get_plain_text())
crypten.print(z_enc.get_plain_text())

In [ ]:
num_features = 100
num_train_examples = 1000
num_test_examples = 100
epochs = 100
lr = 1e-4

features = torch.randn(num_features, num_train_examples)
w_true = torch.randn(1, num_features)
b_true = torch.rand(1)

labels = w_true.matmul(features).add(b_true).sign()
test_features = torch.rand(num_features, num_test_examples)
test_labels = w_true.matmul(test_features).add(b_true).sign()

# gets the first 50 features 
features[:50].size()

#gets the last 50 features
features[50:].size()


In [ ]:
# Lets load in the MNIST Digit Dataset, and Iris Dataset
from sklearn import datasets
import pandas as pd
import numpy as np

iris_dataset = datasets.load_iris()
iris_dataset.keys()


iris_dataset["target"] = iris_dataset["target"][:, np.newaxis]
iris_dataset["target"] = iris_dataset["target"].astype(np.float64)
iris_data = np.concatenate([iris_dataset["data"], iris_dataset["target"]], axis=-1)
iris_dataset["feature_names"].extend(["label"])

iris_cols = iris_dataset["feature_names"]
iris_df = pd.DataFrame(iris_data, columns=iris_cols)

In [ ]:
iris_df.head()

In [ ]:
iris_df["label"].nunique()

In [ ]:
from sklearn.model_selection import train_test_split
X = iris_df.drop(columns=["label"])
y = iris_df[["label"]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
one_hot_labels = torch.eye(3)
print(one_hot_labels)

one_hot_labels[y_train.values.squeeze()]


In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        super().__init__()
        self.features = torch.tensor(features.values)
        
        one_hot_labels = torch.eye(3)
        self.labels = one_hot_labels[labels.values.squeeze()]

    def __len__(self):
        return len(self.labels)


    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


train_dataset = Dataset(X_train, y_train)
test_dataset = Dataset(X_test, y_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, num_workers=1, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, num_workers=1)

In [ ]:
train_dataset[0][0]

In [ ]:
class BasicLinearModel(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU()
        )

        self.l2 = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(hidden_features, out_features),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        out = self.l1(x)
        out = self.l2(out)

        return out

In [ ]:
class Trainer:
    def __init__(self, 
                 input_shape, 
                 model, 
                 train_dataloader, 
                 test_dataloader, 
                 loss_function) -> None:
        
        model_plaintext = model
        dummy_input = torch.empty(*input_shape)
        model = crypten.nn.from_pytorch(model_plaintext, dummy_input)

        self.model = model
        self.model.encrypt()

        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader
        self.loss_function = loss_function
        self.divider = "-" * 4
    

    def train_one_epoch(self, lr:int) -> None:

        # Puts the model into train mode
        self.model.train()

        print("\n" + self.divider + "Train" + self.divider)
        train_loss = 0

        for _, (X, y) in enumerate(self.train_dataloader):
                
            encryped_X = crypten.cryptensor(X)
            encryped_y = crypten.cryptensor(y, requires_grad=True)

            output = self.model(encryped_X)
            loss = self.loss_function(output, encryped_y)

            self.model.zero_grad()
            loss.backward()
            self.model.update_parameters(lr)

            train_loss += loss.get_plain_text()
        
        print(f"Train Batch Loss: {train_loss.item() / len(self.train_dataloader) : .4f}")
    
    def validate(self) -> None:
        self.model.eval()

        print("\n" + self.divider + "Test" + self.divider)
        test_loss = 0

        with torch.no_grad():
            for _, (X, y) in enumerate(self.test_dataloader): 
                encryped_X = crypten.cryptensor(X)
                encryped_y = crypten.cryptensor(y, requires_grad=False)

                output = self.model(encryped_X)
                loss = self.loss_function(output, encryped_y)

                test_loss += loss.get_plain_text()
        
        print(f"Test Batch Loss: {test_loss.item() / len(self.test_dataloader) : .4f}\n")
    

    def run(self, lr, epochs):
        for i in range(epochs):
            print(f"Current Epoch: {i + 1}\n" + "=" * 16)
            self.train_one_epoch(lr)
            self.validate()




In [ ]:
linear_model = BasicLinearModel(in_features=4,
                                hidden_features=10, 
                                out_features=3)

cross_entropy_loss_fn = crypten.nn.CrossEntropyLoss()

trainer = Trainer(model=linear_model,
                  input_shape=(1, 4),
                  train_dataloader=train_dataloader,
                  test_dataloader=test_dataloader,
                  loss_function=cross_entropy_loss_fn)

trainer.run(lr=0.1, 
              epochs=15)


In [ ]:
import crypten.mpc as mpc
import crypten.communicator as comm

private_model = crypten.nn.from_pytorch(linear_model, torch.empty(1, 4))

@mpc.run_multiprocess(world_size=2)
def inference():
    
    private_model.encrypt()
    rank = comm.get().get_rank()
    
    print(f"\nRank {rank}:\n {private_model.state_dict()}")

inference()



In [ ]:
from torchvision.models.resnet import resnet18
import torch

import crypten
import crypten.mpc as mpc

crypten.init()


model = resnet18()
dummmy_input = torch.empty(64, 3, 28, 28)

crypten.nn.from_pytorch(model, dummmy_input)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Sequential):
    """
    Adaptation of LeNet that uses ReLU activations
    """

    # network architecture:
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # Batchnorm results in 
        self.batchnorm1 = crypten.nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.batchnorm1(self.conv2(x))))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # nn.AdaptiveAvgPool2d((1,1))
        return x

In [ ]:
import crypten.nn as cnn
import torch.nn.functional as F

class CryptenLeNet(crypten.nn.Module):
    """
    Adaptation of LeNet that uses ReLU activations
    """

    # network architecture:
    def __init__(self):
        super().__init__()
        self.conv1 = cnn.Conv2d(3, 6, 5)
        self.pool = cnn.MaxPool2d(2, 2)
        self.conv2 = cnn.Conv2d(6, 16, 5)

        # Batchnorm results in 
        self.batchnorm1 = cnn.BatchNorm2d(16)
        self.fc1 = cnn.Linear(16 * 5 * 5, 120)
        self.fc2 = cnn.Linear(120, 84)
        self.fc3 = cnn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.batchnorm1(self.conv2(x))))
        x = x.view(-1, 16 * 5 * 5)
        x = crypten.nn.Rel.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # nn.AdaptiveAvgPool2d((1,1))
        return x

In [ ]:
import crypten.mpc as mpc

leNet = CryptenLeNet()


@mpc.run_multiprocess(world_size=2)
def inference():

    crypted_data = crypten.rand(1, 3, 32, 32)
    
    leNet.encrypt()
    rank = comm.get().get_rank()
    
    print(f"\nRank {rank}:\n {leNet(crypted_data)}")

inference()



Process Process-10:
Process Process-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/comet/anaconda3/envs/research_py/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/comet/anaconda3/envs/research_py/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/comet/anaconda3/envs/research_py/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/comet/anaconda3/envs/research_py/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/comet/anaconda3/envs/research_py/lib/python3.9/site-packages/crypten/mpc/context.py", line 30, in _launch
    return_value = func(*func_args, **func_kwargs)
  File "/home/comet/anaconda3/envs/research_py/lib/python3.9/site-packages/crypten/mpc/context.py", line 30, in _launch
    return_value = func(*func_args, 

In [ ]:
model = CryptenLeNet()
dummmy_input = torch.empty(1, 3, 32, 32)
model.encrypt()

model.eval()



In [ ]:
import torchvision

model = resnet18().to("cpu")
dummy_input = torch.empty(64, 3, 32, 32)
# model = crypten.load_from_party(preloaded=model, model_class=torchvision.models.resnet.ResNet)
model.eval()

crypten.common.serial.register_safe_class(torchvision.models.resnet.ResNet)

model = cnn.from_pytorch(model, dummy_input)

In [ ]:
crypten.common.serial.register_safe_class(model)

In [ ]:
model = resnet18().to("cpu")
dummmy_input = torch.empty(1, 3, 28, 28)


model = crypten.load_from_party(preloaded=model, model_class=resnet18())
crypten.nn.from_pytorch(model, dummmy_input)

In [ ]:
class NoopContextManager:
    """Context manager that does nothing."""

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        pass

context_manager = NoopContextManager()



In [ ]:
import torchvision.models as models
model_name="resnet18"
dummmy_input = torch.empty(1, 3, 224, 224)

with context_manager:
        model = getattr(models, model_name)(pretrained=True)
        model.eval()
        # dataset = datasets.ImageNet(imagenet_folder, split="val", download=download)


encrypted_model = crypten.nn.from_pytorch(model, dummy_input=dummy_input)
encrypted_model.encrypt()

In [6]:

import torch
import torch.nn as nn

import crypten
import crypten.mpc as mpc
import crypten.communicator as comm

import time 

crypten.init()


@mpc.run_multiprocess(world_size=2)
def inference():
    
    data = torch.tensor([-1,1,2])
    encrypted_data = crypten.cryptensor(data)

    regular_relu = nn.ReLU()

    crypted_relu = crypten.nn.ReLU()
    crypted_relu.encrypt()

    rank = comm.get().get_rank()
    start = time.time()
    encrypted_output = crypted_relu(encrypted_data)
    print(f"Crypted RELU: {time.time() - start : 4f}  ")    
    
    start = time.time()
    output = regular_relu(data)

    print(f"RELU: {time.time() - start : 4f}  ")
    print(f"\nRank {rank}:\n Encrypted Output: {encrypted_output} | Real Output: {encrypted_output.get_plain_text()}")
    print(f"\nRank {rank}:\n Output: {output} | Output: {output}")
    

inference()



Crypted RELU:  0.250626  
Crypted RELU:  0.281813  RELU:  0.000209  

RELU:  0.000290  

Rank 1:
 Encrypted Output: MPCTensor(
	_tensor=tensor([4238793319733541260, -496873920919160655, 4651535131736646381])
	plain_text=HIDDEN
	ptype=ptype.arithmetic
) | Real Output: tensor([0., 1., 2.])
Rank 0:
 Encrypted Output: MPCTensor(
	_tensor=tensor([-4238793319733541260,   496873920919226191, -4651535131736515309])
	plain_text=HIDDEN
	ptype=ptype.arithmetic
) | Real Output: tensor([0., 1., 2.])


Rank 0:
 Output: tensor([0, 1, 2]) | Output: tensor([0, 1, 2])

Rank 1:
 Output: tensor([0, 1, 2]) | Output: tensor([0, 1, 2])


[None, None]

1.0